In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from time import sleep
import pandas as pd
import requests
import pymongo
from pprint import pprint

In [2]:
#setting up path to chrome driver for splinter
executable_path={'executable_path':'chromedriver.exe'}

In [3]:
#define function to remove ending substring from string
def remove_substring(string, substring):
    if substring in string:
        return string[:-len(substring)]
    return string

In [4]:
# from fake_useragent import UserAgent
# ua = UserAgent()
# ua.chrome

In [5]:
#creating headers (from https://stackoverflow.com/questions/27652543/how-to-use-python-requests-to-fake-a-browser-visit)
# headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36'}
# headers = {'User-Agent': 'Mozilla/5.0 (X11; CrOS i686 3912.101.0) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/27.0.1453.116 Safari/537.36'}

In [6]:
#url with latest NASA stories
# url_1 = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
# response=requests.get(url_1, headers=headers)

#set up parser
# soup = bs(response.content, 'lxml')

#get latest news articles
# date_latest=soup.find('div', class_='list_date').text
# title_latest=soup.find('div', class_='content_title').text
# summary_latest=soup.find('div', class_='article_teaser_body').text


In [7]:
#url with latest NASA stories
browser = Browser('chrome', **executable_path, headless=False)
url_1 = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'
browser.visit(url_1)

#set up parser
html = browser.html
soup = bs(html, 'lxml')

#get latest news articles
date_latest=soup.find('div', class_='list_date').text
title_latest=soup.find('div', class_='content_title').text
summary_latest=soup.find('div', class_='article_teaser_body').text

#close browser
browser.quit()

News_dict={"Date":date_latest,
           "Title": title_latest,
           "Summary": summary_latest}

In [8]:
pprint(News_dict)

{'Date': 'January 28, 2019',
 'Summary': 'The Mars rover is moving on and has taken its last selfie on the '
            "ridge it's explored for more than a year.",
 'Title': "Curiosity Says Farewell to Mars' Vera Rubin Ridge"}


In [9]:
#getting featured image (with splinter)
browser = Browser('chrome', **executable_path, headless=False)
url_2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url_2)

#clicking the FULL IMAGE BUTTON
browser.find_by_xpath('//*[@id="full_image"]').click()
#clicking the expand button
browser.find_by_xpath('//*[@id="fancybox-lock"]/div/div[1]/a[3]').click()

#set up parser
html = browser.html
soup = bs(html, 'lxml')

#find featured image source
featured_image=soup.find('img', class_='fancybox-image')
featured_image_url='https://www.jpl.nasa.gov'+featured_image['src']
featured_image_title=soup.find('div', class_='fancybox-title')

#removing substring (from a tag) from title
try:
    featured_image_title_full=featured_image_title.text
    featured_image_title_a=featured_image_title.div.text
    featured_image_title=remove_substring(featured_image_title_full, featured_image_title_a)
except:
    featured_image_title=featured_image_title_full

#close browser
browser.quit()

Featured_Img_Dict={"Title": featured_image_title,
                   "Source": featured_image_url}

In [10]:
pprint(Featured_Img_Dict)

{'Source': 'https://www.jpl.nasa.gov/spaceimages/images/mediumsize/PIA16105_ip.jpg',
 'Title': 'Layers at the Base of Mount Sharp'}


In [11]:
#getting featured image (without splinter)
url_2 = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
response=requests.get(url_2)

#set up parser
soup = bs(response.content, 'lxml')

featured_image=soup.find("div", class_="carousel_items")
featured_image_title=featured_image.article['alt']
#have to remove url string in brackets from html code
featured_image_url='https://www.jpl.nasa.gov'+(featured_image.article['style'])[(featured_image.article['style']).find("(")+2:(featured_image.article['style']).find(")")-1]

Featured_Img_Dict={"Title": featured_image_title,
                   "Source": featured_image_url}

In [12]:
pprint(Featured_Img_Dict)

{'Source': 'https://www.jpl.nasa.gov/spaceimages/images/wallpaper/PIA16105-1920x1200.jpg',
 'Title': 'Layers at the Base of Mount Sharp'}


In [13]:
#visitng MarsWeather Twitter page
url_3='https://twitter.com/marswxreport?lang=en'
response=requests.get(url_3)

#set up parser
soup=bs(response.text, 'lxml')

#finding latest tweet for Mars weather
mars_weather=soup.find('div', class_='js-tweet-text-container')

#dividing into p and a tags (sometimes a tag does not exist) and removing a tag from ptag with remove substring function
try:
    mars_weather_p=mars_weather.p.text
    mars_weather_a=mars_weather.a.text
    mars_weather_today=remove_substring(mars_weather_p, mars_weather_a)
except:
    mars_weather_today=mars_weather_p

Mars_weather_dict={"Mars Weather": mars_weather_today}

In [14]:
pprint(Mars_weather_dict)

{'Mars Weather': 'Sol 2302 (2019-01-27), high -7C/19F, low -73C/-99F, pressure '
                 'at 8.15 hPa, daylight 06:47-18:54'}


In [15]:
#mars facts
url_4='https://space-facts.com/mars/'
facts=pd.read_html(url_4)
mars_facts=facts[0]
mars_facts.columns=['Mars Planet Profile', 'Fact Value']

mars_facts_dict=mars_facts.set_index('Mars Planet Profile').to_dict()['Fact Value']

In [16]:
pprint(mars_facts_dict)

{'Equatorial Diameter:': '6,792 km',
 'First Record:': '2nd millennium BC',
 'Mass:': '6.42 x 10^23 kg (10.7% Earth)',
 'Moons:': '2 (Phobos & Deimos)',
 'Orbit Distance:': '227,943,824 km (1.52 AU)',
 'Orbit Period:': '687 days (1.9 years)',
 'Polar Diameter:': '6,752 km',
 'Recorded By:': 'Egyptian astronomers',
 'Surface Temperature:': '-153 to 20 °C'}


In [17]:
#faces of mars images
# url_5='https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars'

#alternate website used because of government shutdown
url_5='http://www.planetary.org/blogs/guest-blogs/bill-dunford/20140203-the-faces-of-mars.html'
response=requests.get(url_5)

#setup parser
soup=bs(response.text, 'lxml')

#parse to get images
hemispheres=soup.find_all('img', class_='img840')
hemisphere_pic_dict={}


#loop through images and get picture link and title of pic and put in dictionary (remove extra space at end of str if exists and 'Mars: ' from beginning)
for hemisphere in hemispheres:
    title=hemisphere['alt']
    image_src=hemisphere['src']
    if title[-1]==' ':
        title=title[:-len(title[-1])]
    if title[0:6]=='Mars: ':
        title=title[6:]
    hemisphere_pic_dict[title]= image_src

In [18]:
pprint(hemisphere_pic_dict)

{'Cerberus Hemisphere': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_cerberus_enhanced_f840.jpg',
 'Schiaparelli Hemisphere': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_schiaparelli_enhanced_f840.jpg',
 'Syrtis Major Hemisphere': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_syrtis_major_enhanced_f840.jpg',
 'Valles Marineris Hemisphere': 'https://planetary.s3.amazonaws.com/assets/images/4-mars/2014/20140202_valles_marineris_enhanced_f840.jpg'}


In [19]:
#Create Connection to MongoDB
conn="mongodb://localhost:27017"
client=pymongo.MongoClient(conn)

In [20]:
#create MarsDB database on MongoDB
MarsDB = client["MarsDB"]

In [21]:
#inserting the following dictionaries into MongoDB database MarsDB
# News_dict
# Featured_Img_Dict
# Mars_weather_dict
# mars_facts_dict
# hemisphere_pic_dict

#drop collections if exist
try:
    MarsDB.drop_collection('marsNews')
    MarsDB.drop_collection('featuredImage')
    MarsDB.drop_collection('marsFacts')
    MarsDB.drop_collection('marsWeather')
    MarsDB.drop_collection('marsHemiPics')
except:
    pass
    
#creating collections
Mars_News=MarsDB['marsNews']
Featured_Img=MarsDB['featuredImage']
Mars_Facts=MarsDB['marsFacts']
Mars_Weather=MarsDB['marsWeather']
Mars_Hemi_Pics=MarsDB['marsHemiPics']

#inserting dictionaries into empty collections
Mars_News.insert(News_dict)
Featured_Img.insert(Featured_Img_Dict)
Mars_Facts.insert(mars_facts_dict)
Mars_Weather.insert(Mars_weather_dict)
Mars_Hemi_Pics.insert(hemisphere_pic_dict)


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:26: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:27: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


ObjectId('5c52105e306bc53e9cdfd4f4')